In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os


In [2]:
#checkpoint function
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:

def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [7]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size
# // 연산자 : 나누기 결과가 int

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6144, train_loss: 2.33491802, train_accuracy: 0.1484, test_Accuracy: 0.1208
Epoch: [ 0] [    1/  468] time: 0.7709, train_loss: 2.31757927, train_accuracy: 0.1250, test_Accuracy: 0.1028
Epoch: [ 0] [    2/  468] time: 0.9240, train_loss: 2.29447794, train_accuracy: 0.0625, test_Accuracy: 0.1028
Epoch: [ 0] [    3/  468] time: 1.0936, train_loss: 2.28304529, train_accuracy: 0.1797, test_Accuracy: 0.1405
Epoch: [ 0] [    4/  468] time: 1.3489, train_loss: 2.29149127, train_accuracy: 0.0781, test_Accuracy: 0.1010
Epoch: [ 0] [    5/  468] time: 1.5414, train_loss: 2.28438902, train_accuracy: 0.0938, test_Accuracy: 0.1010
Epoch: [ 0] [    6/  468] time: 1.7678, train_loss: 2.28141928, train_accuracy: 0.1016, test_Accuracy: 0.1010
Epoch: [ 0] [    7/  468] time: 1.9623, train_loss: 2.26259184, train_accuracy: 0.0859, test_Accuracy: 0.1010
Epoch: [ 0] [    8/  468] time: 2.1468,

Epoch: [ 0] [   75/  468] time: 13.4985, train_loss: 0.97344398, train_accuracy: 0.7734, test_Accuracy: 0.7754
Epoch: [ 0] [   76/  468] time: 13.6636, train_loss: 0.98548102, train_accuracy: 0.8047, test_Accuracy: 0.7764
Epoch: [ 0] [   77/  468] time: 13.8311, train_loss: 0.81358546, train_accuracy: 0.8359, test_Accuracy: 0.7723
Epoch: [ 0] [   78/  468] time: 13.9987, train_loss: 0.94788003, train_accuracy: 0.7734, test_Accuracy: 0.7706
Epoch: [ 0] [   79/  468] time: 14.1882, train_loss: 0.99821901, train_accuracy: 0.7422, test_Accuracy: 0.7751
Epoch: [ 0] [   80/  468] time: 14.3453, train_loss: 0.91873574, train_accuracy: 0.7891, test_Accuracy: 0.7785
Epoch: [ 0] [   81/  468] time: 14.5208, train_loss: 0.93839079, train_accuracy: 0.7656, test_Accuracy: 0.7811
Epoch: [ 0] [   82/  468] time: 14.6864, train_loss: 0.87162709, train_accuracy: 0.8125, test_Accuracy: 0.7861
Epoch: [ 0] [   83/  468] time: 14.8639, train_loss: 0.91947162, train_accuracy: 0.7734, test_Accuracy: 0.7909
E

Epoch: [ 0] [  150/  468] time: 26.2350, train_loss: 0.46271327, train_accuracy: 0.8672, test_Accuracy: 0.8661
Epoch: [ 0] [  151/  468] time: 26.3876, train_loss: 0.50046456, train_accuracy: 0.8750, test_Accuracy: 0.8704
Epoch: [ 0] [  152/  468] time: 26.5791, train_loss: 0.42155054, train_accuracy: 0.9062, test_Accuracy: 0.8732
Epoch: [ 0] [  153/  468] time: 26.7616, train_loss: 0.44242832, train_accuracy: 0.8906, test_Accuracy: 0.8749
Epoch: [ 0] [  154/  468] time: 26.9162, train_loss: 0.44706821, train_accuracy: 0.8672, test_Accuracy: 0.8782
Epoch: [ 0] [  155/  468] time: 27.0728, train_loss: 0.45033368, train_accuracy: 0.8828, test_Accuracy: 0.8799
Epoch: [ 0] [  156/  468] time: 27.2393, train_loss: 0.50058150, train_accuracy: 0.8906, test_Accuracy: 0.8813
Epoch: [ 0] [  157/  468] time: 27.4044, train_loss: 0.58320564, train_accuracy: 0.8203, test_Accuracy: 0.8793
Epoch: [ 0] [  158/  468] time: 27.5560, train_loss: 0.42320442, train_accuracy: 0.8828, test_Accuracy: 0.8776
E

Epoch: [ 0] [  225/  468] time: 38.1575, train_loss: 0.30466706, train_accuracy: 0.9219, test_Accuracy: 0.9045
Epoch: [ 0] [  226/  468] time: 38.3211, train_loss: 0.36913890, train_accuracy: 0.9141, test_Accuracy: 0.9039
Epoch: [ 0] [  227/  468] time: 38.5395, train_loss: 0.40591967, train_accuracy: 0.8672, test_Accuracy: 0.9034
Epoch: [ 0] [  228/  468] time: 38.6921, train_loss: 0.37525946, train_accuracy: 0.9219, test_Accuracy: 0.9027
Epoch: [ 0] [  229/  468] time: 38.8686, train_loss: 0.38133544, train_accuracy: 0.8828, test_Accuracy: 0.9023
Epoch: [ 0] [  230/  468] time: 39.0382, train_loss: 0.38432479, train_accuracy: 0.8984, test_Accuracy: 0.9029
Epoch: [ 0] [  231/  468] time: 39.2287, train_loss: 0.36076090, train_accuracy: 0.8906, test_Accuracy: 0.9036
Epoch: [ 0] [  232/  468] time: 39.3942, train_loss: 0.28637028, train_accuracy: 0.9141, test_Accuracy: 0.9052
Epoch: [ 0] [  233/  468] time: 39.5618, train_loss: 0.28546363, train_accuracy: 0.9141, test_Accuracy: 0.9059
E

Epoch: [ 0] [  299/  468] time: 51.0569, train_loss: 0.36699706, train_accuracy: 0.9062, test_Accuracy: 0.9103
Epoch: [ 0] [  300/  468] time: 51.4549, train_loss: 0.31313509, train_accuracy: 0.9219, test_Accuracy: 0.9087
Epoch: [ 0] [  301/  468] time: 51.6613, train_loss: 0.33187398, train_accuracy: 0.8750, test_Accuracy: 0.9064
Epoch: [ 0] [  302/  468] time: 51.9037, train_loss: 0.32907605, train_accuracy: 0.9062, test_Accuracy: 0.9047
Epoch: [ 0] [  303/  468] time: 52.1296, train_loss: 0.35551071, train_accuracy: 0.9141, test_Accuracy: 0.9051
Epoch: [ 0] [  304/  468] time: 52.3471, train_loss: 0.25470635, train_accuracy: 0.9219, test_Accuracy: 0.9053
Epoch: [ 0] [  305/  468] time: 52.5815, train_loss: 0.30235964, train_accuracy: 0.8906, test_Accuracy: 0.9075
Epoch: [ 0] [  306/  468] time: 52.8069, train_loss: 0.21984047, train_accuracy: 0.9531, test_Accuracy: 0.9096
Epoch: [ 0] [  307/  468] time: 52.9989, train_loss: 0.31889057, train_accuracy: 0.8906, test_Accuracy: 0.9106
E

Epoch: [ 0] [  374/  468] time: 64.7209, train_loss: 0.40487865, train_accuracy: 0.8984, test_Accuracy: 0.9165
Epoch: [ 0] [  375/  468] time: 64.8734, train_loss: 0.27336329, train_accuracy: 0.9219, test_Accuracy: 0.9166
Epoch: [ 0] [  376/  468] time: 65.0430, train_loss: 0.19242266, train_accuracy: 0.9609, test_Accuracy: 0.9159
Epoch: [ 0] [  377/  468] time: 65.2001, train_loss: 0.27334005, train_accuracy: 0.9219, test_Accuracy: 0.9165
Epoch: [ 0] [  378/  468] time: 65.3616, train_loss: 0.35155278, train_accuracy: 0.9141, test_Accuracy: 0.9167
Epoch: [ 0] [  379/  468] time: 65.5093, train_loss: 0.28231391, train_accuracy: 0.9141, test_Accuracy: 0.9180
Epoch: [ 0] [  380/  468] time: 65.6678, train_loss: 0.24379730, train_accuracy: 0.9453, test_Accuracy: 0.9188
Epoch: [ 0] [  381/  468] time: 65.8214, train_loss: 0.33094963, train_accuracy: 0.8906, test_Accuracy: 0.9183
Epoch: [ 0] [  382/  468] time: 65.9910, train_loss: 0.36862546, train_accuracy: 0.8906, test_Accuracy: 0.9193
E

Epoch: [ 0] [  448/  468] time: 77.3057, train_loss: 0.23479459, train_accuracy: 0.9375, test_Accuracy: 0.9221
Epoch: [ 0] [  449/  468] time: 77.4753, train_loss: 0.15985431, train_accuracy: 0.9609, test_Accuracy: 0.9227
Epoch: [ 0] [  450/  468] time: 77.6553, train_loss: 0.25959402, train_accuracy: 0.9375, test_Accuracy: 0.9230
Epoch: [ 0] [  451/  468] time: 77.8229, train_loss: 0.29310092, train_accuracy: 0.8906, test_Accuracy: 0.9235
Epoch: [ 0] [  452/  468] time: 77.9834, train_loss: 0.20823672, train_accuracy: 0.9531, test_Accuracy: 0.9241
Epoch: [ 0] [  453/  468] time: 78.1410, train_loss: 0.26160687, train_accuracy: 0.9375, test_Accuracy: 0.9250
Epoch: [ 0] [  454/  468] time: 78.3714, train_loss: 0.26634613, train_accuracy: 0.9219, test_Accuracy: 0.9247
Epoch: [ 0] [  455/  468] time: 78.5380, train_loss: 0.31154060, train_accuracy: 0.8984, test_Accuracy: 0.9238
Epoch: [ 0] [  456/  468] time: 78.6965, train_loss: 0.26928079, train_accuracy: 0.9219, test_Accuracy: 0.9237
E